In [1]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
from scipy.optimize import fsolve
import ipywidgets as widgets
from IPython.display import display
from thermo_models.RaoultsLawModel import *
from distillation.DistillationModel import DistillationModel
from utils.AntoineEquation import * 
from IPython.display import clear_output
from tqdm import tqdm
from distillation.DistillationTernary import *
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interactive, Button, VBox, HBox, Output
from IPython.display import clear_output


In [2]:
# Antoine Parameters for benzene
Ben_A = 4.72583
Ben_B = 1660.652
Ben_C = -1.461

# Antoine Parameters for toluene
Tol_A = 4.07827
Tol_B = 1343.943
Tol_C = -53.773

# Antoine Parameters for Xylene
Xyl_A = 4.14553
Xyl_B = 1474.403
Xyl_C = -55.377

P_sys = 1.0325
# Create Antoine equations for benzene and toluene
benzene_antoine = AntoineEquationBase10(Ben_A, Ben_B, Ben_C)
toluene_antoine = AntoineEquationBase10(Tol_A, Tol_B, Tol_C)
xylene_antoine = AntoineEquationBase10(Xyl_A, Xyl_B, Xyl_C)

print(benzene_antoine.get_boiling_point(1))
print(toluene_antoine.get_boiling_point(1))
print(xylene_antoine.get_boiling_point(1))
# Create a Raoult's law object
TolBenXylSys = RaoultsLawModel(3,P_sys,[benzene_antoine, toluene_antoine, xylene_antoine])
# TolBenXylSys.plot_ternary_txy(100, keep_zero=0)

352.8600135066221
383.31052448955074
411.0379166982268


In [3]:
R1_slider =  widgets.FloatLogSlider(value=1.0, base=10, min=0, max=2, step=0.001,  description='Reflux Ratio:', readout_format='.1f')
go_button = widgets.Button(description='Create Model', width='15%', margin='0 10px 0 0')
out_oplines = Output()

def plot_rect_strip_ex2(R = R1_slider):
    plt.close('all')  # Ensure that any existing figures are closed before plotting a new one.
    xF = np.array([0.3, 0.3, 0.4])
    xD = np.array([0.990, 0.0100, 1e-10])
    xB = np.array([1e-5, 0.4261,0.5749])
    distillation_model = DistillationModelTernary(TolBenXylSys, xF = xF, xD = xD, xB = xB, reflux = R)
    fig, ax = plt.subplots(1,1,figsize= (5,5))
    distillation_model.plot_rect_strip_comp(ax)
    plt.subplots_adjust(hspace=0)
    plt.show()

In [4]:
def on_go_button_click(b):
    with out_oplines:
        clear_output(wait=True)
        plt.close('all')  # Close all open matplotlib plots
        plot_rect_strip_ex2( R=R1_slider.value)
        
go_button = Button(description='Generate Plot')
go_button.on_click(on_go_button_click)

layout = VBox([HBox([R1_slider, go_button]), out_oplines])
display(layout)

In [5]:
from ipywidgets import HBox, VBox, FloatSlider, Output
import matplotlib.pyplot as plt
import numpy as np

out_residue = Output()

def update_slider2(*args):
    slider2.max = 1 - slider1.value

def update_slider1(*args):
    slider1.max = 1 - slider2.value

slider1 = FloatSlider(value=0.33, min=0, max=1, step=0.01, description="Component 1:")
slider2 = FloatSlider(value=0.33, min=0, max=1 - slider1.value, step=0.01, description="Component 2:")

slider1.observe(update_slider2, 'value')
slider2.observe(update_slider1, 'value')

def plot_residue(change):
    xD = np.array([slider1.value, slider2.value, 1 - (slider1.value + slider2.value)])
    xF = np.array([0.3, 0.3, 0.4])
    xB = np.array([1e-5, 0.4261, 0.5749])
    distillation_model = DistillationModelTernary(TolBenXylSys, xF=xF, xD=xD, xB=xB, reflux=1000)
    with out_residue:
        out_residue.clear_output(wait = True)
        plt.close('all')
        fig, ax = plt.subplots(1, 1, figsize=(5, 5))
        distillation_model.plot_rect_comp(ax)
        plt.subplots_adjust(hspace=0)
        plt.show()

slider1.observe(plot_residue, 'value')
slider2.observe(plot_residue, 'value')

layout = VBox([HBox([slider1, slider2]), out_residue])
display(layout)


/Users/panwapromtep/Library/CloudStorage/OneDrive-JohnsHopkins/ctei_ext_distillation/src/distillation/DistillationModel.py:57: RuntimeWarning: divide by zero encountered in double_scalars
  self.boil_up = ((self.reflux+self.q)*((self.xF[0]-self.xB[0])/(self.xD[0]-self.xF[0]))) + self.q - 1 #this one need 1 component
